Derivative approximation (p. 43, ex. 5.4, variant 1).

In [1]:
import numpy as np
from math import sin, exp
from scipy import misc
from mpmath import diff
import pandas as pd 

# Task 1

Approximately count values.

Making methods which approximately count derivatives.

In [2]:
# Разность вперёд.
def first_derivative_forward_first_error(f, h, x):
    return (f(x + h) - f(x))/h
    
# Разность назад.
def first_derivative_backward_first_error(f, h, x):
    return (f(x) - f(x - h))/h  

# Симметричная разность.
def first_derivative_symmetrical_second_error(f, h, x):
    return (f(x + h) - f(x - h))/(2*h)  

# Разность вперёд, O(h^2).
def first_derivative_forward_second_error(f, h, x):
    return ((-3)*f(x) + 4*f(x + h) - f(x + 2*h))/(2*h)
    
# Разность назад, O(h^2).
def first_derivative_backward_second_error(f, h, x):
    return (3*f(x) - 4*f(x - h) + f(x - 2*h))/(2*h)  

# Симметричная разность второй производной, O(h^2).
def second_derivative_symmetrical_second_error(f, h, x):
    return (f(x + h) - 2*f(x) + f(x - h))/(h**2)  

Initial data.

In [3]:
start = 0
end = 1
step = 0.1
function = lambda x: sin(x)
accuracy = 4 # Знаков после запятой.

Counting values in columns.

In [4]:
# x.
x_list = list(map(lambda x: round(x, accuracy), np.arange(start, end + step, step)))

# f(x).
f_x_list = [round(function(i), accuracy) for i in x_list]

# f'(x).
f_derivative = [round(misc.derivative(function, i,  dx=1e-5), accuracy) for i in x_list] 

# f' O(h)
f_first_derivative_first_error = [round(first_derivative_forward_first_error(function, step, i), accuracy) for i in x_list[:-1]]
f_first_derivative_first_error.append(round(first_derivative_backward_first_error(function, step, x_list[-1]), accuracy))

# погр. O(h).
def list_subtraction(x, y):
    return list(map(lambda a, b: round(abs(a - b), accuracy), x, y))
f_first_error = list_subtraction(f_derivative, f_first_derivative_first_error)

# f' O(h^2).
f_first_derivative_second_error = [round(first_derivative_forward_second_error(function, step, i), accuracy) for i in x_list[:-1]]
f_first_derivative_second_error.append(round(first_derivative_backward_second_error(function, step, x_list[-1]), accuracy))

# погр. O(h^2).
f_second_error = list_subtraction(f_derivative, f_first_derivative_second_error)

# f''(x).
f_second_derivative = [round(misc.derivative(function, i,  dx=1e-5, n=2), accuracy) for i in x_list] 

# f'' O(h^2).
f_second_derivative_error = [round(second_derivative_symmetrical_second_error(function, step, i), accuracy) for i in x_list[1:-1]]
f_second_derivative_error.insert(0, '-')
f_second_derivative_error.append('-')

# погр. O(h^2).
f_second_derivative_second_error = list_subtraction(f_second_derivative[1:-1], f_second_derivative_error[1:-1])
f_second_derivative_second_error.insert(0, '-')
f_second_derivative_second_error.append('-')

Building dataframe.

In [5]:
df = pd.DataFrame()
column_names = ["x", "f(x)", "f'(x)", "f' O(h)", "погр. O(h)", "f' O(h^2)", "погр. O(h^2)", "f''(x)", "f'' O(h^2)", "погр. O(h^2)"]

df[column_names[0]] = x_list
df[column_names[1]] = f_x_list
df[column_names[2]] = f_derivative
df[column_names[3]] = f_first_derivative_first_error
df[column_names[4]] = f_first_error
df[column_names[5]] = f_first_derivative_second_error
df[column_names[6]] = f_second_error
df[column_names[7]] = f_second_derivative
df[column_names[8]] = f_second_derivative_error
df[column_names[9]] = f_second_derivative_second_error

display(df.style.hide_index())

x,f(x),f'(x),f' O(h),погр. O(h),f' O(h^2),погр. O(h^2),f''(x),f'' O(h^2)
0,0,1,0.9983,0.0017,1.0033,-,0,-
0.1,0.0998,0.995,0.9884,0.0066,0.9983,0,-0.0998,-0.0998
0.2,0.1987,0.9801,0.9685,0.0116,0.9833,0.0002,-0.1987,-0.1985
0.3,0.2955,0.9553,0.939,0.0163,0.9584,0.0002,-0.2955,-0.2953
0.4,0.3894,0.9211,0.9001,0.021,0.924,0.0003,-0.3894,-0.3891
0.5,0.4794,0.8776,0.8522,0.0254,0.8804,0.0004,-0.4794,-0.479
0.6,0.5646,0.8253,0.7958,0.0295,0.8279,0.0004,-0.5646,-0.5642
0.7,0.6442,0.7648,0.7314,0.0334,0.7672,0.0005,-0.6442,-0.6437
0.8,0.7174,0.6967,0.6597,0.037,0.6988,0.0006,-0.7174,-0.7168
0.9,0.7833,0.6216,0.5814,0.0402,0.6235,0.0006,-0.7833,-0.7827


# Task 2

Count step experimentally and theoretically.

Initial data.

In [20]:
point = 1
step = 0.1
step_loss = 0.5
step_num = 6
function = lambda x: round(exp(2*x), 5)
function_derivative = lambda x: round(2*exp(2*x), 5)
accuracy = 5e-6 # Знаков после запятой.

In [23]:
# Exact derivateive value.
f_derivative = function_derivative(point)

Посчитаем значения в строчках.

In [24]:
# Шаги h.
steps = [step*step_loss**(i) for i in range(step_num)]

# Приближённые значения производной f' пор. O(h^2).
approx_values = [round(first_derivative_forward_second_error(function, h, point), 4) for h in steps]

# Погрешность
error = [f_derivative - value for value in approx_values]

Building dataframe.

In [26]:
df = pd.DataFrame()
df = df.reindex(columns = range(step_num + 1))


row_names = ["h", "f_x пор. O(h^2)", "погр."]
df.loc[1] = [row_names[0]] + steps
df.loc[2] = [row_names[1]] + approx_values
df.loc[3] = [row_names[2]] + error

df.set_index(0, inplace=True)
display(df)

,1,2,3,4,5,6
0,,,,,,
h,0.10000,0.05000,0.02500,0.01250,0.00625,0.003125
f_x пор. O(h^2),14.54840,14.72490,14.76500,14.77440,14.77680,14.774400
погр.,0.22971,0.05321,0.01311,0.00371,0.00131,0.003710
